# 1. 모델링
**EDA과정에서 얻은 정보를 모델링에 반영**<BR>
1.기대이익 계산 공식을 보면 survival_time == 64 or amount_spent == 0 일 경우 기대이익은 0이 되는 것을 알 수 있다.<BR>
2.EDA과정에서 survival_time가 amount_spent를 예측할때 중요한 feature가 되는 것을 알 수 있다.<BR><BR>


**모델링 과정**<BR>
1.survival_time == 64 과 amount_spent == 0를 우선 예측하여 feature로 사용할 예정이다.(기대이익이 최대한 0이 되는 경우를 피해야 할 것로 판단하였다.)<BR>
2.survivaltime 예측<BR>
3.amount_spent 예측<BR><BR>
  
**XGBoost Parameter Tuning**<BR>
-Parameter Tuning 진행할때 오랜 시간이 걸리는 문제가 있어서 아래 링크를 참조하여 단계별로 GridSearch를 진행하여 문제를 해결하였다.<BR>
-https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/<BR>

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [0]:
import basic_preprocessing

In [0]:
import score_function as sf

## 1. amount_spent 0, !=0 구분

In [0]:
train = basic_preprocessing.getTrainData()

In [0]:
train['target_is_amount_spent'] = 0
train.loc[train['target_amount_spent'] == 0, 'target_is_amount_spent'] = 1

In [0]:
is_amount_df = train.drop(['target_amount_spent','target_survival_time','acc_id'], axis=1)

X_train_is_amount = is_amount_df.drop('target_is_amount_spent', axis=1).values
label_is_amount = is_amount_df['target_is_amount_spent'].values

X_train, X_test, Y_train, Y_test = train_test_split(
  X_train_is_amount,
  label_is_amount,
  random_state= 0 
)

In [7]:
is_amount_df.shape

(40000, 137)

In [8]:
xgbClassifier = xgb.XGBClassifier()
"""

* 시간이 오래걸려서 주석처리
* 아래는 얻은 파라메터 값


param_grid = {
    'silent' : [1],
    'max_depth' : [1,2,4,6,8,10],
    'min_child_weight' : [1,2,4,6,8,10],
    'gamma' : [0.0, 0.1, 0.2, 0.3, 0.4],
    'n_estimators' : [10, 50, 100, 300],
    'seed' : [1],
    'tree_method' : ['gpu_hist']
}

gird_search = GridSearchCV(
    xgbClassifier,
    param_grid,
    cv = 5,
    return_train_score=True,
    n_jobs=8
)

gird_search.fit(X_train, Y_train)

gird_search.best_params_

{
  'gamma': 0.0,
  'max_depth': 4,
  'min_child_weight': 2,
  'n_estimators': 100,
  'seed': 1,
  'silent': 1,
  'tree_method': 'gpu_hist'
}
"""

"\n\n* 시간이 오래걸려서 주석처리\n* 아래는 얻은 파라메터 값\n\n\nparam_grid = {\n    'silent' : [1],\n    'max_depth' : [1,2,4,6,8,10],\n    'min_child_weight' : [1,2,4,6,8,10],\n    'gamma' : [0.0, 0.1, 0.2, 0.3, 0.4],\n    'n_estimators' : [10, 50, 100, 300],\n    'seed' : [1],\n    'tree_method' : ['gpu_hist']\n}\n\ngird_search = GridSearchCV(\n    xgbClassifier,\n    param_grid,\n    cv = 5,\n    return_train_score=True,\n    n_jobs=8\n)\n\ngird_search.fit(X_train, Y_train)\n\ngird_search.best_params_\n\n{\n  'gamma': 0.0,\n  'max_depth': 4,\n  'min_child_weight': 2,\n  'n_estimators': 100,\n  'seed': 1,\n  'silent': 1,\n  'tree_method': 'gpu_hist'\n}\n"

In [0]:
isAmountSpentXGB = xgb.XGBClassifier(
    gamma = 0.0,
    max_depth = 4,
    min_child_weight = 2,
    n_estimators = 100,
    seed = 1,
    tree_method = 'gpu_hist'
).fit(
    X_train_is_amount,
    label_is_amount
)

In [10]:
print('isAmountSpentXGB : 총 {}명 중 {:.2f}% 정확도로 0, !=0 을 구분'.format(
    label_is_amount.shape[0], 100 * metrics.accuracy_score(
        isAmountSpentXGB.predict(X_train_is_amount),
        label_is_amount
    )
))

isAmountSpentXGB : 총 40000명 중 85.67% 정확도로 0, !=0 을 구분


## 2. survival_time 1~63, 64 구분

In [0]:
del(train)
train = basic_preprocessing.getTrainData()

train['target_is_survival_time'] = 0
train.loc[train['target_survival_time'] == 64, 'target_is_survival_time'] = 1

is_survival_df = train.drop(['target_survival_time','target_amount_spent','acc_id','target_is_amount_spent'], axis=1)

X_train_is_survival = is_survival_df.drop('target_is_survival_time', axis=1).values
label_is_survival = is_survival_df['target_is_survival_time'].values

X_train, X_test, Y_train, Y_test = train_test_split(
  X_train_is_survival,
  label_is_survival,
  random_state= 0 
)

In [13]:
xgbClassifier = xgb.XGBClassifier()
"""
param_grid = {
    'max_depth' : [1,2,4,6,8,10],
    'min_child_weight' : [1,2,4,6,8,10],
    'gamma' : [0.0, 0.1, 0.2, 0.3, 0.4],
    'n_estimators' : [10, 50, 100, 300],
    'seed' : [1],
    'tree_method' : ['gpu_hist']
#     'learning_rate' = [.01, .03, .05, .1, .25]
}

gird_search = GridSearchCV(
    xgbClassifier,
    param_grid,
    cv = 5,
    return_train_score=True,
    n_jobs=8
)

gird_search.fit(X_train, Y_train)

gird_search.best_score_

gird_search.score(X_test, Y_test)

gird_search.best_params_
{
  'gamma': 0.0,
  'max_depth': 10,
  'min_child_weight': 10,
  'n_estimators': 300,
  'seed': 1,
  'silent': 1,
  'tree_method': 'gpu_hist'
}
"""


"\nparam_grid = {\n    'max_depth' : [1,2,4,6,8,10],\n    'min_child_weight' : [1,2,4,6,8,10],\n    'gamma' : [0.0, 0.1, 0.2, 0.3, 0.4],\n    'n_estimators' : [10, 50, 100, 300],\n    'seed' : [1],\n    'tree_method' : ['gpu_hist']\n#     'learning_rate' = [.01, .03, .05, .1, .25]\n}\n\ngird_search = GridSearchCV(\n    xgbClassifier,\n    param_grid,\n    cv = 5,\n    return_train_score=True,\n    n_jobs=8\n)\n\ngird_search.fit(X_train, Y_train)\n\ngird_search.best_score_\n\ngird_search.score(X_test, Y_test)\n\ngird_search.best_params_\n{\n  'gamma': 0.0,\n  'max_depth': 10,\n  'min_child_weight': 10,\n  'n_estimators': 300,\n  'seed': 1,\n  'silent': 1,\n  'tree_method': 'gpu_hist'\n}\n"

In [0]:
isSurvivalTimeXGB = xgb.XGBClassifier(
    gamma = 0.0,
    max_depth = 10,
    min_child_weight = 10,
    n_estimators = 300,
    seed = 1,
    tree_method = 'gpu_hist'
).fit(
    X_train_is_survival,
    label_is_survival
)

In [15]:
print('isSurvivalTimeXGB : 총 {}명 중 {:.2f}% 정확도로 0~63, 64를 구분'.format(
    label_is_survival.shape[0], 100 * metrics.accuracy_score(
        isSurvivalTimeXGB.predict(X_train_is_survival),
        label_is_survival
    )
))

isSurvivalTimeXGB : 총 40000명 중 96.45% 정확도로 0~63, 64를 구분


## 3. survival_time 1~63, 64 구분값과 amount_spent 0, !=0 구분 값을 feature로 사용한 최종 모델링

In [0]:
train = basic_preprocessing.getTrainData()

In [0]:
train_df = train.drop(['target_survival_time','target_amount_spent','acc_id','target_is_survival_time','target_is_amount_spent'], axis=1)
isAmountPredict = isAmountSpentXGB.predict(train_df.values)
isSurvivalPredict = isSurvivalTimeXGB.predict(train_df.values)

train_df['is_amount_spent'] = isAmountPredict
train_df['is_survival_time'] = isSurvivalPredict

In [0]:
# train_df['acc_id'] = train['acc_id']
# train_df['target_survival_time'] = train['target_survival_time']

In [0]:
train_label = train[['target_survival_time']]

X_train, X_test, Y_train, Y_test = train_test_split(
  train_df.values,
  train_label.values,
  random_state= 0 
)

In [25]:
sruvivalTimeXGB = xgb.XGBRegressor(
    max_depth=10,
    min_child_weight=6,
    gamma=0,
    n_estimators = 300,
    seed = 1,
    tree_method = 'gpu_hist'
).fit(
  train_df.values,
  train_label.values
)

[04:32:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [0]:
train_df['survival_time'] = np.round(sruvivalTimeXGB.predict(train_df.values)).astype(np.int32)

# 1~64의 결과가 나오도록 수정
train_df.loc[train_df['survival_time'] <= 0, 'survival_time'] = 1
train_df.loc[train_df['survival_time'] >= 65, 'survival_time'] = 64

In [51]:
print('XGB : 총 {}명 중 {:.2f}% 정확도로 0~64를 구분'.format(
    train_label.values.shape[0], 100 * metrics.accuracy_score(
        np.round(sruvivalTimeXGB.predict(train_df.drop('survival_time', axis=1).values)).astype(np.int32),
        train_label.values
    )
))

XGB : 총 40000명 중 30.04% 정확도로 0~64를 구분


In [53]:
print('XGBoosting, RMRS : {}'.format(
    np.sqrt(metrics.mean_squared_error(
        np.round(sruvivalTimeXGB.predict(train_df.drop('survival_time', axis=1).values)).astype(np.int32),
        train_label.values
    )
)))

XGBoosting, RMRS : 4.194880808795406


## 4. amount_spent 모델링

In [0]:
train_label = train[['target_amount_spent']]

X_train, X_test, Y_train, Y_test = train_test_split(
  train_df.values,
  train_label.values,
  random_state= 0 
)

In [56]:
amountSpentXgbBest = xgb.XGBRegressor(
    max_depth=1,
    min_child_weight=1,
    seed=27,
    gamma=0,
    n_estimators=300,
    tree_method= 'gpu_hist'
).fit(
  train_df.values,
  train_label.values
)

[05:10:20] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [58]:
print('amount_spent XGBoosting, RMRS : {}'.format(
    np.sqrt(metrics.mean_squared_error(
        np.round(amountSpentXgbBest.predict(train_df.values)).astype(np.int32),
        train_label.values
    )
)))

amount_spent XGBoosting, RMRS : 0.7009644539715771


## 5. 예측 결과

In [0]:
train_testing = pd.DataFrame(
    {
        "acc_id" : train['acc_id'].tolist(),
        "survival_time" : train_df['survival_time'].tolist(),
        "amount_spent" : train['amount_spent'].tolist()
    }
)

In [0]:
trainLabel = basic_preprocessing.getTrainLabelData()

In [68]:
print("기대이익 : {}".format(sf.score_function(train_testing, trainLabel)))

기대이익 : 29053.660814440183
